## 1 操作系统准备

$\quad$VMware 12中安装Windows XP系统

## 2. 获取和安装Python

$\quad$《Gray Hat Python》一书中使用的是Python2.5，可以直接安装集成了python2.5版本的Immunity调试器即可。

$\quad$由于Python2.5版本太老了，可能后续某些python库安装不上的话，直接下载安装Python2.7版本备用

#### $\quad$注:将所有使用到的工具(VMware虚拟机和Windows XP镜像除外)打包，并上传百度云网盘备份

## 3.配置开发环境

$\quad\spadesuit\quad$查看本书源码使用Sublime 3，编辑代码使用Atom，代码管理使用msysgit(物理机)

$\quad\spadesuit\quad$执行代码Shell使用msysgit(虚拟机)

$\quad\spadesuit\quad$安装VMTools，物理机创建共享文件夹并映射为虚拟机的网络驱动器，实现虚拟机与物理机之间通信

### 3.1 The Hacker’s Best Friend: ctypes

The Python module $\text{ctypes}$ is by far one of the most powerful libraries available to the Python developer. The $\text{ctypes}$ library enables you to call functions in dynamically linked libraries and has extensive capabilities for creating complex C datatypes and utility functions for low-level memory manipulation. It is essential that you understand the basics of how to use the ctypes library, as you will be relying on it heavily throughout the book.  
#### 注:查看Python官方文档学习熟悉ctypes库的使用

### 3.2 Using Dynamic Libraries

The first step in utilizing $\text{ctypes}$ is to understand how to resolve and access functions in a dynamically linked library. A $\text{dynamically linked library}$ is a compiled binary that is linked at runtime to the main process executable. On Windows platforms these binaries are called $\text{dynamic link libraries (DLL)}$, and on Linux they are called $\text{shared objects (SO)}$. In both cases, these binaries expose functions through exported names, which get resolved to actual addresses in memory. Normally at runtime you have to resolve the function addresses in order to call the functions; however, with $\text{ctypes}$ all of the dirty work is already done.

There are three different ways to load dynamic libraries in ctypes: $\textrm{cdll()}$, $\textrm{windll()}$, and $\textrm{oledll()}$. The difference among all three is in the way the functions inside those libraries are called and their resulting return values. The $\textrm{cdll()}$ method is used for loading libraries that export functions using the standard $cdecl$ calling convention. The $\textrm{windll()}$ method loads libraries that export functions using the $stdcall$ calling convention, which is the native convention of the $\text{Microsoft Win32 API}$. The $\textrm{oledll()}$ method operates exactly like the $\textrm{windll()}$ method; however, it assumes that the exported functions return a $\text{Windows HRESULT}$ error code, which is used specifically for error messages returned from Microsoft $\text{Component Object Model (COM)}$ functions.

For a quick example you will resolve the $\text{printf()}$ function from the $\text{C}$ runtime on both Windows and Linux and use it to output a test message. On Windows the C runtime is $\text{msvcrt.dll}$, located in C:\WINDOWS\system32\,and on Linux it is $\text{libc.so.6}$, which is located in /lib/ by default. Create a $\texttt{printf.py}$ script in your normal Python working directory, and enter the following code.

In [ ]:
# printf.py

from ctypes import *

msvcrt          = cdll.msvcrt
message_string  = "Hello world!\n"

msvcrt.printf("Testing: %s", message_string)

The following is the output of this script:

### 3.3 Constructing C Datatypes

Creating a C datatype in Python is just downright sexy, in that nerdy, weird way. Having this feature allows you to fully integrate with components written in C and C++, which greatly increases the power of Python. Briefly review Table 1-1 to understand how datatypes map back and forth between C, Python, and the resulting ctypes type.

See how nicely the datatypes are converted back and forth? Keep this table handy in case you forget the mappings. The ctypes types can be initialized with a value, but it has to be of the proper type and size. For a demonstration, open your Python shell and enter some of the following examples:

The last example describes how to assign the variable $\text{seitz}$ a character pointer to the string $\text{"loves the python"}$. To access the contents of that pointer use the $\text{seitz.value}$ method, which is called $dereferencing$ a pointer.

### 3.4 Passing Parameters by Reference

It is common in C and C++ to have a function that expects a pointer as one of its parameters. The reason is so the function can either write to that location in memory or, if the parameter is too large, pass by value. Whatever the case may be, $\text{ctypes}$ comes fully equipped to do just that, by using the $\textrm{byref()}$ function. When a function expects a pointer as a parameter, you call it like this: $\text{function_main( byref(parameter) )}$.

### 3.5 Defining Structures and Unions

Structures and unions are important datatypes, as they are frequently used throughout the Microsoft Win32 API as well as with libc on Linux. A $\text{structure}$ is simply a group of variables, which can be of the same or different datatypes. You can access any of the member variables in the structure by using dot notation, like this: $\text{beer_recipe.amt_barley}$. This would access the $\text{amt_barley}$ variable contained in the $\text{beer_recipe}$ structure. Following is an example of defining a structure (or struct as they are commonly called) in both C and Python.

### In C

In [ ]:
struct beer_recipe
{
    int amt_barley;
    int amt_water;
};

### In Python

In [6]:
class beer_recipe(Structure):
    _fields_ = [
    ("amt_barley", c_int),
    ("amt_water", c_int),
    ]

As you can see, $\text{ctypes}$ has made it very easy to create C-compatible structures. Note that this is not in fact a complete recipe for beer, nor do I encourage you to drink barley and water.

Unions are much the same as structures. However, in a union all of the member variables share the same memory location. By storing variables in this way, unions allow you to specify the same value in different types. The next example shows a union that allows you to display a number in three different ways.

### In C

In [ ]:
union {
    long barley_long;
    int barley_int;
    char barley_char[8];
}barley_amount;

### In Python

In [7]:
class barley_amount(Union):
    _fields_ = [
    ("barley_long", c_long),
    ("barley_int", c_int),
    ("barley_char", c_char * 8),
    ]

If you assigned the $\text{barley_amount}$ union’s member variable $\text{barley_int}$ a value of 66, you could then use the $\text{barley_char}$ member to display the character representation of that number. To demonstrate, create a new file called $\texttt{unions.py}$ and hammer out the following code.

In [8]:
# unions.py

from ctypes import *

class barley_amount(Union):
    _fields_ = [
    ("barley_long", c_long),
    ("barley_int", c_int),
    ("barley_char", c_char * 8),
    ]

value = raw_input("Enter the amount of barley to put into the beer vat:")
my_barley = barley_amount(int(value))

print "Barley amount as a long: %ld" % my_barley.barley_long
print "Barley amount as an int: %d"  % my_barley.barley_long
print "Barley amount as a char: %s"  % my_barley.barley_char

Enter the amount of barley to put into the beer vat:66
Barley amount as a long: 66
Barley amount as an int: 66
Barley amount as a char: B


As you can see, by assigning the union a single value, you get three different representations of that value. If you are confused by the output of the $\text{barley_char}$ variable, $B$ is the $\text{ASCII}$ equivalent of decimal 66.

The $\text{barley_char}$ member variable is an excellent example of how to define an array in $\text{ctypes}$. In $\text{ctypes}$ an array is defined by multiplying a type by the number of elements you want allocated in the array. In the previous example, an eight-element character array was defined for the member variable $\text{ctypes}$.